In [108]:
from tqdm import tqdm
from common_methods import *
from context_decoder import ContextDecoder
import os
import json

In [109]:
decoder = ContextDecoder("mistralai/Mistral-7B-Instruct-v0.2")

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [110]:
def load_data():
    try:
        with open("datasets/final_dataset.json", 'r') as file:
            data = json.load(file)
    
    except FileNotFoundError:
        print(f"Error: File not found at {file_path}")
    except json.JSONDecodeError:
        print(f"Error: Invalid JSON format in {file_path}")
    except Exception as e:
        print(f"An unexpected error occurred: {str(e)}")
        
    return data

In [111]:
data[0].keys()

dict_keys(['prompt', 'short_answers', 'long_answer'])

In [112]:
def generate_completions(alpha=2.0,max_tokens=50):
    data = load_data()
    
    for point in data:
        prompt = point['prompt']
        prompt_nc = prompt.split("\n\nQuestion: ")[-1]
        
        out_base = decoder.regular_decoding(prompt,max_tokens=max_tokens,debug=False,show_tqdm=True)
        out_cad = decoder.context_aware_decoding(prompt,prompt_nc,alpha=alpha,max_tokens=max_tokens,debug=False,show_tqdm=True)

        point['baseline_completion'] = out_base
        point['cad_completion'] = out_cad
        print(prompt_nc,point['short_answers'],out_cad,"***",out_base)

    with open("runs/completions.json", 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=4)

In [114]:
# generate_completions()